In [1]:
import pandas as pd
import os
import glob

In [2]:
import sys 
sys.path.append('../code')

In [3]:
%load_ext autoreload
%autoreload 2

from libs import io 
from libs import constants as cons

[2026-01-29 15:27:51] Error: The file ../../../LLMCaller/config/llm_setup.json was not found.
Available LLMs: (24): llama-3.3-8b llama-4-scout llama-4-mav gpt-oss-20b gpt-oss-120b qwen3-8b qwen3-14b qwen3-32b qwen3-30b-a3b-2507 qwen3-235b-a22b-2507 gemma-3-12b gemma-3-27b mistral-small-3.2-24b mistral-medium-3 llama-3.1-70b llama-3.3-70b llama-3.1-405b grok-4-fast deepseek-chat-v3.1 deepseek-r1-0528 gemini-2.5-flash gemini-2.5-flash-grounded gemini-2.5-pro gemini-2.5-pro-grounded


In [4]:
APS_OA_DATA_TAR_GZ = '../../APS/data/final_dataset.tar.gz'
FN_COAUTHORS = '../results/organised_data/aps_coauthor_networks.json'
FN_EDGELIST = '../results/augmented_aps/coauthorships.csv'

# Option 1

In [5]:
from itertools import combinations
from tqdm import tqdm

In [7]:
df_authorships = io.read_file_from_tar_gz_as_dataframe(APS_OA_DATA_TAR_GZ, cons.APS_OA_AUTHORSHIPS_FN)
df_authorships.rename(columns={'id_author': 'id_author_oa'}, inplace=True)
df_authorships.head()

,id_publication,id_author_oa,position,id_institution,authorship_flag
0,2155512932,5061426463,first,4.210121e+09,original_openalex
1,2155512932,5064510725,middle,1.282106e+09,original_openalex
2,2155512932,5111938223,middle,4.210121e+09,original_openalex
3,2155512932,5052336272,middle,4.210121e+09,original_openalex
4,2155512932,5110594767,middle,4.210121e+09,original_openalex


In [8]:
edges = []
groups = df_authorships.groupby('id_publication')
for g, tmp in tqdm(groups, total=groups.ngroups, desc="Processing coauthorships"):

    # we sort the ids, for easier check up of undirected edges
    ids = tmp.id_author_oa.sort_values().unique()
    pairs = list(combinations(ids, 2))

    edges.extend(pairs)


Processing coauthorships: 100%|██████████| 678788/678788 [02:08<00:00, 5263.24it/s]


In [9]:
df_edges = io.pd.DataFrame(edges, columns=['src','dst'])
df_edges.shape
# 37 022 942

(37022942, 2)

In [11]:
df_edges.head(5) # 19 049 285

,src,dst
0,5002794296,5003458529
1,5002794296,5044160613
2,5002794296,5046134145
3,5002794296,5061541886
4,5002794296,5103500275


In [10]:
io.save_csv(df_edges, FN_EDGELIST, index=False)
FN_EDGELIST

[2026-01-29 15:33:01] Data successfully saved to ../results/augmented_aps/coauthorships.csv


'../results/augmented_aps/coauthorships.csv'

In [12]:
df_edges.drop_duplicates().shape

(9531006, 2)

In [13]:
!cat ../results/augmented_aps/coauthorships.csv | wc -lc

37022943 814504732


In [14]:
!ls -lh ../results/augmented_aps/

total 1.2G
-rwxrwsr-x 1 espinl llmscholar  33M Aug 21 11:10 authors_aps_stats.csv
-rwxrwsr-x 1 espinl llmscholar  90M Jan 25  2025 authors_demographics.csv
-rwxrwsr-x 1 espinl llmscholar 102M Jan 20  2025 authors_demographics_full.csv
-rwxrwsr-x 1 espinl llmscholar 148M Jan 26  2025 authors_stats.csv
-rw-r--r-- 1 espinl llmscholar 777M Jan 29 15:33 coauthorships.csv
-rwxrwsr-x 1 espinl llmscholar 8.9M Jan 20  2025 ethnicity.csv
-rwxrwsr-x 1 espinl llmscholar 8.9M Jan 20  2025 ethnicity_dx.csv
-rwxrwsr-x 1 espinl llmscholar 8.1M Jan 20  2025 ethnicity_ec.csv
-rwxrwsr-x 1 espinl llmscholar 825K Jan 20  2025 gender_0.csv
-rwxrwsr-x 1 espinl llmscholar 824K Jan 20  2025 gender_1.csv
-rwxrwsr-x 1 espinl llmscholar 824K Jan 20  2025 gender_2.csv
-rwxrwsr-x 1 espinl llmscholar 824K Jan 20  2025 gender_3.csv
-rwxrwsr-x 1 espinl llmscholar 824K Jan 20  2025 gender_4.csv
-rwxrwsr-x 1 espinl llmscholar 824K Jan 20  2025 gender_5.csv
-rwxrwsr-x 1 espinl llmscholar 824K Jan 20  2025 gender_6.csv
-r

# Option 2

In [ ]:
def get_coauthors_and_authors(fn_coauthors):
    # authors = set()
    # edge_list = set()
    edge_df = pd.DataFrame()

    obj_coauthors = {int(item['openalex_id'].replace('A','')):item for item in io.read_list_of_dicts(fn_coauthors)}
    aps_oa_mapping = {obj['id_author']:oa_id for oa_id, obj in obj_coauthors.items()}
    
    sources = []
    targets = []

    for id_author_oa in obj_coauthors.keys():

        id_author_oa = int(id_author_oa)
        oa_coauthors_ids = [aps_oa_mapping[aps_id] for aps_id in obj_coauthors[id_author_oa]['aps_co_authors'] if aps_id not in cons.NONE and pd.notna(aps_id)]
        
        sources.extend([id_author_oa] * len(oa_coauthors_ids))
        targets.extend(oa_coauthors_ids)

        # authors |= set([id_author_oa])
        # authors |= set(oa_coauthors_ids)    
        # edge_list |= set([f'{id_author_oa}\t{coauthor_id}' for coauthor_id in oa_coauthors_ids])
        # edge_df = pd.concat([edge_df, pd.DataFrame({'src': id_author_oa, 'dst': oa_coauthors_ids})])

    edge_df = pd.DataFrame({'src': sources, 'dst': targets})
    return edge_df

In [ ]:
edge_df = get_coauthors_and_authors(FN_COAUTHORS)

In [ ]:
len(edge_list)

In [ ]:
len(authors)

In [ ]:
edge_df.shape

In [ ]:
edge_df.head(10)

In [ ]:
io.validate_path(FN_EDGELIST)
io.save_csv(edge_df, FN_EDGELIST)